# Group Normalization (GN) — Gelişmiş ve Kontrolü Kolay Kullanım (CNN odaklı)

Bu defterde:

1) **Önce kod**: Projelerin çoğunda kullanılabilecek, kontrollü ve güvenli bir **GroupNorm yardımcı modülü** veriliyor.  
2) **Sonra açıklama**: Kodda geçen tüm mekanizmalar tek tek anlatılıyor:
   - `num_groups` nasıl seçilir?
   - `eps`, `affine`, `channels_last` ne işe yarar?
   - GN’nin LN/IN ile ilişkisi
   - CNN bloklarında GN’yi doğru bağlama pratikleri
   - Diğer opsiyonlar: SyncBN, FrozenBN, Weight Standardization, vb.

> Not: Burada bir “tam model” kurmuyoruz. Ama GN’nin **projeye tak-çalıştır** şekilde kullanılması için gereken mekanizmaları kuruyoruz.


---

# 1) KOD (ÖNCE)

Aşağıdaki kod parçası:
- GN’yi **NCHW (channels-first)** CNN akışında sorunsuz kullanır.
- İstersen **NHWC (channels-last)** tensorlerle de kullanabilir.
- `num_groups` için **otomatik seçim** (C % G == 0) yapar.
- İstersen GN’yi **LN-benzeri (G=1)** veya **IN-benzeri (G=C)** moda alabilir.
- Yanlış ayarları erken yakalamak için **doğrulama (validation)** içerir.



In [ ]:
import torch
import torch.nn as nn

# ----------------------------
# 1) Yardımcı: group seçimi
# ----------------------------
def choose_gn_groups(C: int, preferred=(32, 16, 8, 4, 2, 1)) -> int:
    ## “Eğer mümkünse GN’yi 32 grup yap, olmazsa 16, olmazsa 8… en son 1’e düş
    ## Çünkü CNN/detection pratiklerinde “GroupNorm(32)” yıllardır default gibi kullanılıyor
            ## C=24 → 32 gelmez, 16 gelmez, 8 gelir 
            ## C=96 → 32 gelir, 16 da gelir, 8 de gelir
            ## Bazı C’lerde birden fazla seçenek mümkün.
    for g in preferred:
        if g <= C and (C % g == 0):
            return g
    return 1

# ----------------------------
# 2) GN yapılandırması (kontrol kolay)
# ----------------------------
class GNConfig:
    def __init__(
        self,
        groups="auto",               # "auto" veya int
        eps: float = 1e-5,
        affine: bool = True,
        preferred_groups=(32, 16, 8, 4, 2, 1),
        mode: str = "gn"             # "gn" | "ln_like" | "in_like"
    ):
        assert mode in {"gn", "ln_like", "in_like"}
        self.groups = groups
        self.eps = eps
        self.affine = affine
        self.preferred_groups = preferred_groups
        self.mode = mode

# ----------------------------
# 3) Gelişmiş GN modülü (channels_first / channels_last destekli)
# ----------------------------
class GroupNormFlex(nn.Module):
    """Projelerde tekrar kullanılabilir GN wrapper.

    Özellikler:
    - channels_first (NCHW / NCDHW) veya channels_last (NHWC / NDHWC) ile çalışabilir
    - groups='auto' ile C'ye göre mantıklı group seçer
    - mode:
        - 'gn'      : normal GN (G=auto veya int)
        - 'ln_like' : GN(1) -> LN benzeri
        - 'in_like' : GN(C) -> IN benzeri
    """
    def __init__(self, num_channels: int, config: GNConfig | None = None, *, channels_last: bool = False):
        super().__init__()
        if config is None:
            config = GNConfig()
        self.num_channels = int(num_channels)
        self.channels_last = bool(channels_last)
        self.config = config

        # --- groups belirle ---
        if config.mode == "ln_like":
            groups = 1
        elif config.mode == "in_like":
            groups = self.num_channels
        else:
            if config.groups == "auto":
                groups = choose_gn_groups(self.num_channels, config.preferred_groups)
            else:
                # config.groups "auto" değilse, kullanıcı GN’nin grup sayısını manuel vermek istiyor demektir.
                groups = int(config.groups)

        # --- doğrulama ---
        if groups < 1:
            raise ValueError("num_groups >= 1 olmalı")
        if self.num_channels % groups != 0:
            raise ValueError(f"C % G == 0 olmalı. C={self.num_channels}, G={groups}")
        self.groups = groups

        self.gn = nn.GroupNorm(
            num_groups=self.groups,
            num_channels=self.num_channels,
            eps=float(config.eps),
            affine=bool(config.affine)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        ## NHWC yalnızca GN’nin anlayacağı formata (NCHW) geçici olarak çevrilir,
        ## GN uygulanır ve pipeline bozulmasın diye tekrar eski formata döndürülür.
        ## NCHW kullanıyorsan bu adımların hiçbiri çalışmaz bile.

        if self.channels_last: #  Bu şunu soruyor: “Bu katmana gelen tensor NHWC mi, yoksa NCHW mi?”
            # channels_last=False :: NCHW geliyor :: Direkt nn.GroupNorm uygula 
            # channels_last=True :: Tensor NHWC geliyor :: GN uygulayabilmek için geçici olarak NCHW’ye çevir
            
            # 2D: NHWC -> NCHW -> GN -> NHWC
                ## GPU (özellikle Tensor Core) bazı durumlarda NHWC ile daha hızlı

            if x.ndim == 4: 
                # Biz NHWC gelirse ne yapıyoruz, onu konuşuyoruz.
                # NHWC → NCHW  çeviriyoruz 
                x = x.permute(0, 3, 1, 2).contiguous() # Bu sadece ekseni yer değiştirir, veri değişmez.
                ## .contiguous() neden? :: permute sadece view döndürür, memory sıralı değildir. :: Bunu bellekte düzgün sıraya koy
                x = self.gn(x) ## Şimdi GN uygulanabilir :: GN kanalları gruplara bölüyor :: H, W ile birlikte

                return x.permute(0, 2, 3, 1).contiguous() ## Ama neden tekrar NHWC’ye dönüyoruz? 
                ## Bu katmandan önceki katmanlar NHWC idi.Sonraki katmanlar da NHWC bekliyor

            # 3D: NDHWC -> NCDHW -> GN -> NDHWC
            if x.ndim == 5:
                x = x.permute(0, 4, 1, 2, 3).contiguous()
                x = self.gn(x)
                return x.permute(0, 2, 3, 4, 1).contiguous()

            raise ValueError("channels_last=True için x.ndim 4 veya 5 olmalı (NHWC/NDHWC).")

        ## Peki NCHW ise :: O zaman bu karmaşaya HİÇ girmiyoruz:
        return self.gn(x)


# ----------------------------
# 4) (Opsiyonel) Conv -> GN -> Act bloğu (GN entegrasyon örneği)
# ----------------------------
class ConvGNAct(nn.Module):
    def __init__(self, in_ch: int, out_ch: int, k: int = 3, s: int = 1, p=None, act: str = "silu", gn_config: GNConfig | None = None):
        super().__init__()
        if p is None:
            p = k // 2
        self.conv = nn.Conv2d(in_ch, out_ch, kernel_size=k, stride=s, padding=p, bias=False)
        self.norm = GroupNormFlex(out_ch, gn_config, channels_last=False)

        if act == "relu":
            self.act = nn.ReLU(inplace=True)
        elif act == "silu":
            self.act = nn.SiLU(inplace=True)
        elif act == "gelu":
            self.act = nn.GELU()
        elif act in ("none", None):
            self.act = nn.Identity()
        else:
            raise ValueError("act: 'relu' | 'silu' | 'gelu' | 'none'")

    def forward(self, x):
        return self.act(self.norm(self.conv(x)))

In [2]:
x = torch.randn(2, 64, 32, 32)
cfg = GNConfig(groups='auto', mode='gn')
gn = GroupNormFlex(64, cfg)

y = gn(x)
print('NCHW:', x.shape, '->', y.shape, '| groups:', gn.groups)

# LN-benzeri: GN(1)
gn_ln = GroupNormFlex(64, GNConfig(mode='ln_like'))
y2 = gn_ln(x)
print('LN-like groups:', gn_ln.groups)

# IN-benzeri: GN(C)
gn_in = GroupNormFlex(64, GNConfig(mode='in_like'))
y3 = gn_in(x)
print('IN-like groups:', gn_in.groups)


NCHW: torch.Size([2, 64, 32, 32]) -> torch.Size([2, 64, 32, 32]) | groups: 32
LN-like groups: 1
IN-like groups: 64


In [3]:
x_nhwc = torch.randn(2, 32, 32, 64)  # (N,H,W,C)
gn_cl = GroupNormFlex(64, GNConfig(groups=32), channels_last=True)
y_nhwc = gn_cl(x_nhwc)
print('NHWC:', x_nhwc.shape, '->', y_nhwc.shape, '| groups:', gn_cl.groups)


NHWC: torch.Size([2, 32, 32, 64]) -> torch.Size([2, 32, 32, 64]) | groups: 32


---

# 2) AÇIKLAMA (SONRA)

Aşağıda kodun her parçası ve GN mekanizması **tek tek** anlatılıyor.


## 2.1 `choose_gn_groups(C)`: Neden gerekli?

GN’de kritik kural:
- **C % G == 0** olmalı (kanallar gruplara eşit bölünecek)

Projelerde kanal sayıları farklı olabilir (32, 64, 96, 128, 192, ...).  
Bu fonksiyon:
- önce **32 grup** dener (en yaygın pratik)
- olmazsa 16/8/4/2/1 diye iner

Böylece “her yerde çalışan” bir GN ayarı sağlar.


## 2.2 `GNConfig`: Neden config objesi?

Projelerde norm ayarları sık değişir:
- `groups`: 32 mi? auto mu?
- `eps`: fp16 için 1e-5 mi 1e-6 mı?
- `affine`: açık mı kapalı mı?
- “LN-benzeri” mi istiyorum, “IN-benzeri” mi?

Bunları her yerde parametre geçirmek yerine `GNConfig` ile:
- tek yerden yönetirsin
- deney/ablation yaparken kolayca değiştirirsin


## 2.3 `GroupNormFlex`: Asıl yeniden kullanılabilir GN katmanı

Bu sınıfın amacı:
- Tek satırla GN oluşturmak
- Yanlış ayarı erkenden yakalamak
- Channels-last (NHWC) gelirse otomatik dönüştürmek

### 2.3.1 `mode` seçenekleri

- `mode="gn"`  
  Normal GroupNorm: `G=auto` veya manuel `G=int`

- `mode="ln_like"`  
  **GN(1)**: tüm kanalları tek grup yapar ⇒ LN benzeri davranış

- `mode="in_like"`  
  **GN(C)**: her kanalı ayrı grup yapar ⇒ IN benzeri davranış


## 2.4 `channels_last`: NHWC ile GN kullanımı

PyTorch `nn.GroupNorm`, varsayılan olarak **channels-first** (NCHW) bekler.

Ama bazı projelerde tensor channels-last olabilir (NHWC):
- hız optimizasyonu
- belirli kernel’ler
- bazı custom pipeline’lar

`GroupNormFlex(channels_last=True)`:
- NHWC -> NCHW permute eder
- GN uygular
- tekrar NHWC’ye döner

> Eğer projen tamamen NCHW ise `channels_last=False` bırak ve maliyet ekleme.


## 2.5 `eps` neden var?

GN formülü:
\[
\hat{x} = \frac{x - \mu}{\sqrt{\sigma^2 + \epsilon}}
\]

`eps`:
- sayısal stabilite içindir
- `var` çok küçük olursa bölme problemini önler
- fp16/mixed-precision eğitimde bazen önem kazanır

Pratik:
- çoğu zaman `1e-5` iyi default
- bazı durumlarda `1e-6` denenebilir


## 2.6 `affine` (gamma/beta) ne işe yarar?

GN normalize ettikten sonra genelde:
\[
y = \gamma \hat{x} + \beta
\]

- `affine=True`: `gamma` ve `beta` öğrenilir (default)
- `affine=False`: sadece normalize eder

Pratikte çoğu CNN’de `affine=True` tercih edilir:
- ağın ifade gücünü azaltmaz
- norm sonrası yeniden ölçekleme esnekliği verir


## 2.7 CNN’de GN’yi doğru bağlama: `ConvGNAct`

CNN pratiğinde yaygın pattern:

**Conv2d -> GroupNorm -> Activation**

### Bias neden `False`?
- GN’nin affine kısmı (beta) zaten shift yapar
- Conv bias + GN beta çoğu zaman gereksiz parametre
- Bu yüzden pratikte **Conv bias=False** yaygındır


## 2.8 GN hangi senaryolarda çok iyi çalışır?

- **Object detection / segmentation** (batch küçük)
- VRAM limitli eğitim (batch 1–4)
- Fine-tuning sırasında BN istatistikleri karışıyorsa
- Tek GPU / küçük batch ve SyncBN kullanmak istemiyorsan

Genel slogan:
> “Batch küçükse GN güvenli bir default’tur.”


## 2.9 Diğer opsiyonlar ve GN ile ilişkileri (kısa)

### Sync BatchNorm (SyncBN)
- Multi-GPU ile batch istatistiğini GPU’lar arasında senkronlar
- Efektif batch büyür
- Eğer altyapı uygunsa BN performansına yaklaşır

### Frozen BatchNorm
- BN running stats güncellenmez (özellikle pretrained backbone fine-tune)
- Bazı detection pipeline’larında yaygın

### Weight Standardization (WS)
- Aktivasyonu değil, **ağırlıkları** normalize eder
- GN ile birlikte kullanıldığında bazen stabilite artar (GN+WS)


---

# 3) Kısa kullanım örnekleri (projeye tak-çalıştır)

### 3.1 Sadece GN katmanı
```python
norm = GroupNormFlex(C, GNConfig(groups='auto'))
```

### 3.2 LN benzeri (GN(1))
```python
norm = GroupNormFlex(C, GNConfig(mode='ln_like'))
```

### 3.3 IN benzeri (GN(C))
```python
norm = GroupNormFlex(C, GNConfig(mode='in_like'))
```

### 3.4 CNN bloğu
```python
block = ConvGNAct(in_ch, out_ch, gn_config=GNConfig(groups='auto'))
```


---

# 4) Özet

- GN, kanalları gruplara bölerek normalize eder ve batch’ten bağımsızdır.
- Small-batch detection/segmentation için çok uygundur.
- `GroupNormFlex` ile:
  - `groups=auto` seçimi
  - LN-like / IN-like modları
  - channels-last desteği
  tek yerde yönetilir.
- CNN’de tipik bağlanış: **Conv -> GN -> Activation** (Conv bias=False).
